# 13.3 最適制御

In [ ]:
#[K,L,Cost,Xa,XSim,CostSim] = 
#   kalman_lqg( A,B,C,C0, H,D,D0, E0, Q,R, X1,S1  [NSim,Init,Niter] )
#
# Compute optimal controller and estimator for generalized LQG
#
# u(t)    = -L(t) x(t)
# x(t+1)  = A x(t) + B (I + Sum(C(i) rnd_1)) u(t) + C0 rnd_n
# y(t)    = H x(t) + Sum(D(i) rnd_1) x(t) + D0 rnd_n
# xhat(t+1) = A xhat(t) + B u(t) + K(t) (y(t) - H xhat(t)) + E0 rnd_n
# x(1)    ~ mean X1, covariance S1
#
# cost(t) = u(t)' R u(t) + x(t)' Q(t) x(t)
#
# NSim    number of simulated trajectories (default 0)  (optional)
# Init    0 - open loop; 1 (default) - LQG; 2 - random  (optional)
# Niter   iterations; 0 (default) - until convergence   (optional)
#
# K       Filter gains
# L       Control gains
# Cost    Expected cost (per iteration)
# Xa      Expected trajectory
# XSim    Simulated trajectories
# CostSim Empirical cost
#
# This is an implementation of the algorithm described in:
#  Todorov, E. (2005) Stochastic optimal control and estimation
#  methods adapted to the noise characteristics of the
#  sensorimotor system. Neural Computation 17(5): 1084-1108
# The paper is available online at www.cogsci.ucsd.edu/~todorov

# Copyright (C) Emanuel Todorov, 2004-2006

In [2]:
using Random

In [3]:
using LinearAlgebra

In [50]:
function kalman_lqg(A, B, C, C0, H, D, D0, E0, Q, R, X1, S1, NSim)
# initialization

# numerical parameters
MaxIter = 500
Eps = 10^-15

# determine sizes
szX = size(A, 1)
szU = size(B, 2)
szY = size(H, 1)
szC = size(C, 3)
szC0 = size(C0, 2)
szD = size(D, 3)
szD0 = size(D0, 2)
szE0 = size(E0, 2)
N = size(Q, 3)

# initialize missing optional parameters
NSim = 0
Init = 1
Niter = 0

# if C or D are scalar, replicate them into vectors
if size(C,1)==1 && szU>1
   C = C*ones(szU,1)
end
    
if length(D[:])==1
    if D[1]==0
        D = zeros(szY,szX)
    else
        D = D*ones(szX,1)
        if szX !=  szY
            println("D can only be a scalar when szX = szY") # assert
        end
    end
end

print("OK")
    
# if C0,D0,E0 are scalar, set them to 0 matrices and adjust size
if length(C0[:])==1 && C0[1]==0
    C0 = zeros(szX,1)
end
if length(D0[:])==1 && D0[1]==0
    D0 = zeros(szY,1)
end
if length(E0[:])==1 && E0[1]==0
    E0 = zeros(szX,1)
end


    
# initialize policy and filter
K = zeros(szX, szY, N-1)
L = zeros(szU, szX, N-1)


#######################################################################
# run iterative algorithm - until convergence or MaxIter

for iter = 1:MaxIter
   # initialize covariances
   SiE = S1
   SiX = X1*X1'
   SiXE = zeros(szX,szX)
   
   # forward pass - recompute Kalman filter   
   for k = 1:N-1
      # compute Kalman gain
      temp = SiE + SiX + SiXE + SiXE'
      if size(D,2)==1
          DSiD = diag(diag(temp).*D.^2)
      else
          DSiD = zeros(szY,szY)
          for i=1:szD
              DSiD = DSiD + D(:,:,i)*temp*D(:,:,i)'
          end
      end
            
      K[:,:,k] = A*SiE*H'*pinv(H*SiE*H' + D0*D0' + DSiD)
      
      # compute new SiE
      newE = E0*E0' + C0*C0' + (A-K[:,:,k]*H)*SiE*A'
      LSiL = L[:,:,k]*SiX*L[:,:,k]'
      if size(C,2)==1
         newE = newE + B*diag(diag(LSiL).*C.^2)*B'
      else
         for i=1:szC
            newE = newE + B*C[:,:,i]*LSiL*C[:,:,i]'*B'
         end
      end
      
      # update SiX, SiE, SiXE
      SiX = (E0*E0' + K[:,:,k]*H*SiE*A' + (A-B*L[:,:,k])*SiX*(A-B*L[:,:,k])' 
                + (A-B*L[:,:,k])*SiXE*H'*K[:,:,k]' + K[:,:,k]*H*SiXE'*(A-B*L[:,:,k])')
      SiE = newE
      SiXE = (A-B*L[:,:,k])*SiXE*(A-K[:,:,k]*H)' - E0*E0'
   end
   
   
   # first pass initialization
   if iter==1
      if Init==0         # open loop
         K = zeros(szX,szY,N-1) 
      elseif Init==2     # random
         K = randn(szX,szY,N-1)
      end
   end
   
   
   # initialize optimal cost-to-go function
   Sx = Q[:,:,N]
   Se = zeros(szX,szX)
   Cost[iter] = 0
   
   # backward pass - recompute control policy
   for k=N-1:-1:1
      
      # update Cost
      Cost[iter] += tr(Sx*C0*C0') + tr(Se*(K[:,:,k]*D0*D0'*K[:,:,k]' + E0*E0' + C0*C0'))
      
      # Controller
      temp = R + B'*Sx*B
      BSxeB = B'*(Sx+Se)*B
      if size(C,2)==1
         temp = temp + diag(diag(BSxeB).*C.^2)
      else
         for i=1:size(C,3)
            temp = temp + C[:,:,i]' * BSxeB * C[:,:,i]
         end
      end
      L[:,:,k] = pinv(temp) * B' * Sx * A
      
      # compute new Se
      newE = A' * Sx * B * L[:,:,k] + (A-K[:,:,k]*H)' * Se * (A-K[:,:,k]*H)
      
      # update Sx and Se
      Sx = Q[:,:,k] + A'*Sx*(A-B*L[:,:,k]);
      KSeK = K[:,:,k]'*Se*K[:,:,k];
      if size(D,2)==1
          Sx = Sx + diag(diag(KSeK).*D.^2)
      else
          for i=1:szD
              Sx = Sx + D[:,:,i]'*KSeK*D[:,:,i]
          end
      end  
      Se = newE
   end
   
   
   # adjust cost
   Cost[iter] += X1'*Sx*X1 + tr((Se+Sx)*S1)     
   
   # progress bar
   if rem(iter, 10)==0
      println('.')
   end
   
   # check convergence of Cost
   if (Niter>0 && iter>=Niter) || (Niter==0 && iter>1 && abs(Cost(iter-1)-Cost(iter))<Eps) || (Niter==0 && iter>20 && sum(diff(dist(iter-10:iter))>0)>3)
      break
   end
end

# print result
if Cost[iter-1] != Cost[iter]
   #println("" Log10DeltaCost = #.2f\n',log10(abs(Cost(iter-1)-Cost(iter))));
else
   println(" DeltaCost = 0")
end;


#######################################################################
# compute average trajectory

Xa = zeros(szX,N)
Xa[:,1] = X1

for k=1:N-1
   u = -L[:,:,k] * Xa[:,k]
   Xa[:,k+1] = A * Xa[:,k] + B*u
end


#######################################################################
# simulate noisy trajectories

if NSim > 0
   
   # square root of S1
   u, s, v = svd(S1);
   sqrtS = u*diag(sqrt(diag(s)))*v'
   
   # initialize
   XSim = zeros(szX,NSim,N)
   Xhat = zeros(szX,NSim,N)
   Xhat[:,:,1] = repeat(X1, outer = [1, NSim]) # repmatから変更
   XSim[:,:,1] = repeat(X1, outer = [1, NSim]) + sqrtS*randn(szX, NSim)
   
   CostSim = 0
   
   # loop over N
   for k=1:N-1
      
      # update control and cost
      U = -L[:,:,k] * Xhat[:,:,k]
      CostSim += sum(sum(U.*(R*U))) + sum(sum(XSim[:,:,k] .* (Q[:,:,k] * XSim[:,:,k])))
      
      # compute noisy control
      Un = U
      if size(C,2)==1
         Un = Un + U.*randn(szU,NSim).* repeat(C, outer = [1, NSim]) #repmat(C,[1,NSim])
      else
         for i=1:szC
            Un = Un + (C[:,:,i]*U).*  repeat(randn(1,NSim), outer = [szU, 1]) #repmat(randn(1,NSim),[szU 1])
         end
      end
      
      # compute noisy observation
      y = H * XSim[:,:,k] + D0*randn(szD0,NSim)
      if size(D,2)==1
         y += XSim[:,:,k] .* randn(szY,NSim) .* repeat(D, outer = [1, NSim])
      else
         for i=1:szD
            y += (D[:,:,i] * XSim[:,:,k]) .* repeat(randn(1,NSim), outer = [szY, 1]) #repmat(randn(1,NSim),[szY 1])
         end
      end
      
      XSim[:,:,k+1] = A*XSim[:,:,k] + B*Un + C0*randn(szC0, NSim)
      Xhat[:,:,k+1] = A*Xhat[:,:,k] + B*U + K[:,:,k] * (y-H*Xhat[:,:,k]) + E0*randn(szE0,NSim)
   end;
   
   # final cost update
   CostSim += sum(sum(XSim[:,:,N] .* (Q[:,:,N] * XSim[:,:,N])))
   CostSim /= NSim
   
    else
       XSim = []
       CostSim = []
    end
    return K,L,Cost,Xa,XSim,CostSim
end

kalman_lqg (generic function with 2 methods)

実装した関数を用いる。

In [4]:
#############################################################
# define parameters

dt = 0.01;        # time step (sec)
m = 1;            # mass (kg)
b = 0;            # damping (N/sec)
tau = 40;         # time constant (msec)
c = 0.5;          # control-dependent noise

r = 0.00001;      # control signal penalty
v = 0.2;          # endpoint velocity penalty
f = 0.02;         # endpoint force penalty

pos = 0.5*0.02;   # position noise
vel = 0.5*0.2;    # velocity noise
frc = 0.5*1.0;    # force noise

N = 30;               # duration in number of time steps
T = 0.1;              # target distance

In [5]:
# compute system dynamics and cost matrices

dtt = dt/(tau/1000);

A = zeros(5,5);
A[1,1] = 1
A[1,2] = dt;
A[2,2] = 1 - dt*b/m
A[2,3] = dt/m
A[3,3] = 1 - dtt
A[3,4] = dtt
A[4,4] = 1 - dtt
A[5,5] = 1

B = zeros(5,1)
B[4,1] = dtt

C = c
C0 = 0


H = zeros(3,5)
H[1:3,1:3] = Matrix{Float64}(I, 3, 3)

D = 0
D0 = diag([pos vel frc])

E0 = 0

R = r/N;
   
Q = zeros(5,5,N)

d = zeros(3,5)
d[1,1] = 1
d[1,5] = -1
d[2,2] = v
d[3,3] = f
Q[:,:,N] = d'*d

X1 = zeros(5,1)
X1[5] = T
S1 = zeros(5,5)

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [6]:
D = 0
E0 = 0
NSim = 10


# numerical parameters
MaxIter = 500
Eps = 10^-15

# determine sizes
szX = size(A, 1)
szU = size(B, 2)
szY = size(H, 1)
szC = size(C, 3)
szC0 = size(C0, 2)
szD = size(D, 3)
szD0 = size(D0, 2)
szE0 = size(E0, 2)
N = size(Q, 3)

# initialize missing optional parameters
NSim = 0
Init = 1
Niter = 0

# if C or D are scalar, replicate them into vectors
if size(C,1)==1 && szU>1
   C = C*ones(szU,1)
end

In [39]:
szY

3

In [17]:
a = 1

1

In [18]:
size(a, 2)

1

In [7]:
if size(D, 2)==1
    if D[1]==0
        D = zeros(szY,szX)
    else
        D = D*ones(szX,1)
        if szX !=  szY
            println("D can only be a scalar when szX = szY") # assert
        end
    end
end
    
# if C0,D0,E0 are scalar, set them to 0 matrices and adjust size
if size(C0, 2)==1 && C0[1]==0
    C0 = zeros(szX,1)
end
if size(D0, 2)==1 && D0[1]==0
    D0 = zeros(szY,1)
end
if size(E0, 2)==1 && E0[1]==0
    E0 = zeros(szX,1)
end

# initialize policy and filter
K = zeros(szX, szY, N-1)
L = zeros(szU, szX, N-1)

1×5×29 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0

...

[:, :, 27] =
 0.0  0.0  0.0  0.0  0.0

[:, :, 28] =
 0.0  0.0  0.0  0.0  0.0

[:, :, 29] =
 0.0  0.0  0.0  0.0  0.0

In [34]:
temp = SiE + SiX + SiXE + SiXE'
DSiD = diag(diag(temp)' * D.^2)

LoadError: DimensionMismatch("second dimension of A, 3, does not match length of x, 5")

In [38]:
(D.^2) * diag(temp)

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

In [46]:
i = 1
D[:,:,i] * temp * D[:,:,i]'

3×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [47]:
#######################################################################
# run iterative algorithm - until convergence or MaxIter

#for iter = 1:MaxIter
iter = 1
# initialize covariances
SiE = S1
SiX = X1*X1'
SiXE = zeros(szX,szX)

# forward pass - recompute Kalman filter   
for k = 1:N-1
    # compute Kalman gain
    temp = SiE + SiX + SiXE + SiXE'

    if size(D,2)==1
        DSiD = diagm((D.^2) * diag(temp))#diagm(diag(temp) .* D.^2)
    else
        DSiD = zeros(szY,szY)
        for i = 1:szD
            DSiD += D[:,:,i] * temp * D[:,:,i]'
        end
    end


    K[:,:,k] = A * SiE * H' * pinv(H*SiE*H' + D0*D0' + DSiD)

    # compute new SiE
    newE = E0 * E0' + C0 * C0' + (A - K[:,:,k] * H) * SiE * A'
    LSiL = L[:,:,k]*SiX*L[:,:,k]'
    if size(C,2)==1
        newE += B * diagm(diag(LSiL) .* C.^2) * B'
    else
        for i=1:szC
            newE += B * C[:,:,i] * LSiL * C[:,:,i]' * B'
        end
    end

    # update SiX, SiE, SiXE
    SiX = E0*E0' + K[:,:,k] * H * SiE * A' + (A - B * L[:,:,k]) * SiX * (A - B * L[:,:,k])' 
    SiX += (A-B*L[:,:,k])*SiXE*H'*K[:,:,k]' + K[:,:,k]*H*SiXE'*(A-B*L[:,:,k])'
    SiE = newE
    SiXE = (A-B*L[:,:,k]) * SiXE * (A - K[:,:,k] * H)' - E0*E0'
end
"""
# first pass initialization
if iter==1
  if Init==0 # open loop
     K = zeros(szX,szY,N-1) 
  elseif Init==2 # random
     K = randn(szX,szY,N-1)
  end
end


# initialize optimal cost-to-go function
Sx = Q[:,:,N]
Se = zeros(szX,szX)
Cost[iter] = 0

# backward pass - recompute control policy
for k=N-1:-1:1

  # update Cost
  Cost[iter] += tr(Sx*C0*C0') + tr(Se*(K[:,:,k]*D0*D0'*K[:,:,k]' + E0*E0' + C0*C0'))

  # Controller
  temp = R + B'*Sx*B
  BSxeB = B'*(Sx+Se)*B
  if size(C,2)==1
     temp = temp + diag(diag(BSxeB).*C.^2)
  else
     for i=1:size(C,3)
        temp = temp + C[:,:,i]' * BSxeB * C[:,:,i]
     end
  end
  L[:,:,k] = pinv(temp) * B' * Sx * A

  # compute new Se
  newE = A' * Sx * B * L[:,:,k] + (A-K[:,:,k]*H)' * Se * (A-K[:,:,k]*H)

  # update Sx and Se
  Sx = Q[:,:,k] + A'*Sx*(A-B*L[:,:,k])
  KSeK = K[:,:,k]'*Se*K[:,:,k]
  if size(D,2)==1
      Sx = Sx + diag(diag(KSeK).*D.^2)
  else
      for i=1:szD
          Sx = Sx + D[:,:,i]'*KSeK*D[:,:,i]
      end
  end  
  Se = newE
end

# adjust cost
Cost[iter] += X1'*Sx*X1 + tr((Se+Sx)*S1)     

# progress bar
if rem(iter, 10)==0
  println('.')
end
# check convergence of Cost
if (Niter>0 && iter>=Niter) || (Niter==0 && iter>1 && abs(Cost(iter-1)-Cost(iter))<Eps) || (Niter==0 && iter>20 && sum(diff(dist(iter-10:iter))>0)>3)
  break
end
"""
#end

LoadError: DimensionMismatch("dimensions must match: a has dims (Base.OneTo(3), Base.OneTo(3)), b has dims (Base.OneTo(1), Base.OneTo(1)), mismatch at 1")

In [1]:
#K, L, Cost, Xa, XSim, CostSim = kalman_lqg(A, B, C, C0, H, 0, D0, 0, Q, R, X1, S1, 10)

In [ ]:
#############################################################
# optimize, plot simulated trajectories

clf;
plot(squeeze(XSim(1,:,:))','r'); hold on;
plot(Xa(1,:),'k','linewidth',2);
xlabel('time step');
ylabel('position');